In [ ]:
import pandas as pd
file_paths = {
    "Dataset 1 (Mendeley)": "Students_Performance_Mendeley.csv",
    "Dataset 2 (Kaggle Exams)": "Students_Performance_kaggle.csv"
}

for name, path in file_paths.items():
    print(f"\n--- {name} ---")
    try:
        df = pd.read_csv(path)
        print("✅ Columns:\n", df.columns.tolist())
        print("📊 Data Types:\n", df.dtypes)
        print("🔍 Preview:\n", df.head(3))
    except Exception as e:
        print(f"❌ Could not load {name}: {e}")



--- Dataset 1 (Mendeley) ---
✅ Columns:
 ['Department', 'Gender', 'HSC', 'SSC', 'Income', 'Hometown', 'Computer', 'Preparation', 'Gaming', 'Attendance', 'Job', 'English', 'Extra', 'Semester', 'Last', 'Overall']
📊 Data Types:
 Department      object
Gender          object
HSC            float64
SSC            float64
Income          object
Hometown        object
Computer         int64
Preparation     object
Gaming          object
Attendance      object
Job             object
English          int64
Extra           object
Semester        object
Last           float64
Overall        float64
dtype: object
🔍 Preview:
                 Department  Gender   HSC   SSC                        Income  \
0  Business Administration    Male  4.17  4.84            Low (Below 15,000)   
1  Business Administration  Female  4.92  5.00  Upper middle (30,000-50,000)   
2  Business Administration    Male  5.00  4.83  Lower middle (15,000-30,000)   

  Hometown  Computer        Preparation             Gaming

In [ ]:
import pandas as pd

kaggle_path = "Students_Performance_kaggle.csv"

try:
    df_kaggle = pd.read_csv(kaggle_path)
    print("✅ Columns:\n", df_kaggle.columns.tolist())
    print("📊 Data Types:\n", df_kaggle.dtypes)
    print("🔍 Preview:\n", df_kaggle.head(3))
except Exception as e:
    print(f"❌ Could not load Kaggle dataset: {e}")


✅ Columns:
 ['gender', 'race/ethnicity', 'parental level of education', 'lunch', 'test preparation course', 'math score', 'reading score', 'writing score']
📊 Data Types:
 gender                         object
race/ethnicity                 object
parental level of education    object
lunch                          object
test preparation course        object
math score                      int64
reading score                   int64
writing score                   int64
dtype: object
🔍 Preview:
    gender race/ethnicity parental level of education     lunch  \
0  female        group B           bachelor's degree  standard   
1  female        group C                some college  standard   
2  female        group B             master's degree  standard   

  test preparation course  math score  reading score  writing score  
0                    none          72             72             74  
1               completed          69             90             88  
2                    non

In [ ]:
import pandas as pd

# Load both datasets
mendeley_df = pd.read_csv("Students_Performance_Mendeley.csv")
kaggle_df = pd.read_csv("Students_Performance_kaggle.csv")

# Standardize column names to make merging easier
mendeley_df.rename(columns={'Gender': 'gender', 'Preparation': 'test preparation course'}, inplace=True)

# Merge the datasets on common columns (gender and test preparation course)
merged_df = pd.merge(mendeley_df, kaggle_df, on=['gender', 'test preparation course'], how='outer')

# Handle missing values (optional, depending on how you want to treat them)
# For categorical columns, fill with 'Missing'
merged_df[['gender', 'test preparation course', 'race/ethnicity', 'parental level of education', 'lunch']] = \
    merged_df[['gender', 'test preparation course', 'race/ethnicity', 'parental level of education', 'lunch']].fillna('Missing')

# For numerical columns, fill with 0
numerical_columns = ['HSC', 'SSC', 'math score', 'reading score', 'writing score', 'Last', 'Overall']
merged_df[numerical_columns] = merged_df[numerical_columns].fillna(0)

# Ensure correct data types for numerical columns
merged_df['HSC'] = merged_df['HSC'].astype(float)
merged_df['SSC'] = merged_df['SSC'].astype(float)
merged_df['math score'] = merged_df['math score'].astype(int)
merged_df['reading score'] = merged_df['reading score'].astype(int)
merged_df['writing score'] = merged_df['writing score'].astype(int)

# Preview the merged dataset
print(merged_df.head())

# Save the new merged dataset
merged_df.to_csv("merged_student_performance.csv", index=False)


                         Department  gender   HSC   SSC  \
0           Business Administration  Female  4.92  5.00   
1           Business Administration  Female  2.19  3.17   
2  Computer Science and Engineering  Female  3.33  4.95   
3  Computer Science and Engineering  Female  4.51  4.75   
4  Computer Science and Engineering  Female  4.58  4.94   

                          Income Hometown  Computer test preparation course  \
0   Upper middle (30,000-50,000)     City       3.0                0-1 Hour   
1   Lower middle (15,000-30,000)  Village       3.0                0-1 Hour   
2  Lower middle (15,000-30,000)      City       3.0                0-1 Hour   
3   Lower middle (15,000-30,000)  Village       1.0                0-1 Hour   
4            High (Above 50,000)  Village       3.0                0-1 Hour   

              Gaming Attendance  ... Extra  Semester   Last Overall  \
0           0-1 Hour   80%-100%  ...   Yes       7th  3.467   3.467   
1          2-3 Hours   80%-1

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

# Load the merged dataset
merged_df = pd.read_csv("merged_student_performance.csv")

# Data Preprocessing: Encode categorical variables
label_encoder = LabelEncoder()

# Identify categorical columns
categorical_columns = ['gender', 'test preparation course', 'race/ethnicity', 'parental level of education',
                       'lunch', 'Department', 'Hometown', 'Job', 'Extra', 'Semester']

# Encoding categorical columns
for col in categorical_columns:
    merged_df[col] = label_encoder.fit_transform(merged_df[col])

# Encoding the 'Income' column by mapping categories to numeric values
income_mapping = {
    'Low (Below 15,000)': 1,
    'Upper middle (30,000-50,000)': 2,
    'High (Above 50,000)': 3,
    'Missing': 0
}
merged_df['Income'] = merged_df['Income'].map(income_mapping)

# Handle time-related columns (e.g., 'Gaming' and 'Attendance') by mapping them to numeric values
time_mapping = {
    '0-1 Hour': 0.5,
    '1-2 Hours': 1.5,
    '2-3 Hours': 2.5,
    'More than 3 Hours': 3
}

merged_df['Gaming'] = merged_df['Gaming'].map(time_mapping)
merged_df['Attendance'] = merged_df['Attendance'].map({'80%-100%': 1, '60%-80%': 0.8, '40%-60%': 0.6, '0%-40%': 0.4})

# Fill missing values with the column mean for numerical columns
merged_df.fillna(merged_df.mean(), inplace=True)

# Features (X) and Target (y)
X = merged_df.drop(columns=['Overall'])  # Dropping target column 'Overall'
y = merged_df['Overall']  # Target column 'Overall'

# Split data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling numerical data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train a Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

# Make predictions
y_pred_lr = lr_model.predict(X_test_scaled)

# Evaluate the Linear Regression model
print("Linear Regression Model Evaluation:")
print(f"Mean Absolute Error: {mean_absolute_error(y_test, y_pred_lr)}")
print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred_lr)}")
print(f"R-squared: {r2_score(y_test, y_pred_lr)}")

# Train a Random Forest Regressor model for comparison
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train_scaled, y_train)

# Make predictions with Random Forest
y_pred_rf = rf_model.predict(X_test_scaled)

# Evaluate the Random Forest model
print("\nRandom Forest Model Evaluation:")
print(f"Mean Absolute Error: {mean_absolute_error(y_test, y_pred_rf)}")
print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred_rf)}")
print(f"R-squared: {r2_score(y_test, y_pred_rf)}")


Linear Regression Model Evaluation:
Mean Absolute Error: 0.060461078761573375
Mean Squared Error: 0.023459389403171424
R-squared: 0.9901894562440284

Random Forest Model Evaluation:
Mean Absolute Error: 0.046214615384615354
Mean Squared Error: 0.024526781290635433
R-squared: 0.9897430808232206


In [ ]:
import pandas as pd

# Load the datasets
mat_data = pd.read_csv("student-mat.csv", sep=";")
pro_data = pd.read_csv("student-por.csv", sep=";")

# Check the first few rows of each dataset to ensure they are loaded correctly
print("Math Dataset Head:")
print(mat_data.head())

print("\nPortuguese Dataset Head:")
print(pro_data.head())

# Merge the datasets (concatenate them)
merged_data = pd.concat([mat_data, pro_data], ignore_index=True)

# Check the columns in the merged dataset
print("\nColumns in the merged dataset:", merged_data.columns)

# Check the first few rows of the merged dataset
print("\nMerged Dataset Head:")
print(merged_data.head())


Math Dataset Head:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  ...  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher  ...   
1     GP   F   17       U     GT3       T     1     1  at_home     other  ...   
2     GP   F   15       U     LE3       T     1     1  at_home     other  ...   
3     GP   F   15       U     GT3       T     4     2   health  services  ...   
4     GP   F   16       U     GT3       T     3     3    other     other  ...   

  famrel freetime  goout  Dalc  Walc health absences  G1  G2  G3  
0      4        3      4     1     1      3        6   5   6   6  
1      5        3      3     1     1      3        4   5   5   6  
2      4        3      2     2     3      3       10   7   8  10  
3      3        2      2     1     1      5        2  15  14  15  
4      4        3      2     1     2      5        4   6  10  10  

[5 rows x 33 columns]

Portuguese Dataset Head:
  school sex  age address famsize Pstatus  

In [ ]:
import pandas as pd

# Load the datasets
mat_data = pd.read_csv("student-mat.csv", sep=";")
pro_data = pd.read_csv("student-por.csv", sep=";")

# Merge the datasets (concatenate them)
merged_data = pd.concat([mat_data, pro_data], ignore_index=True)

# Save the merged dataset to a new CSV file
merged_data.to_csv("merged_student_data.csv", index=False, sep=";")

# Confirm that the CSV file has been saved
print("Merged dataset saved as 'merged_student_data.csv'")


Merged dataset saved as 'merged_student_data.csv'


In [ ]:
import pandas as pd

# Load UCL dataset
ucl = pd.read_csv("merged_student_data.csv", sep=';')

# Load Mendeley+Kaggle merged dataset
mk = pd.read_csv("merged_student_performance.csv")

# Clean and match common fields
ucl = ucl.rename(columns={
    "sex": "gender",
    "activities": "Extra",
    "G1": "math score",
    "G2": "reading score",
    "G3": "writing score"
})

# Add missing columns in UCL to match Mendeley+Kaggle
for col in mk.columns:
    if col not in ucl.columns:
        ucl[col] = "Missing"

# Add missing columns in mk to match UCL (for academic + social info)
for col in ucl.columns:
    if col not in mk.columns:
        mk[col] = "Missing"

# Ensure same column order
ucl = ucl[mk.columns]

# Concatenate datasets
final = pd.concat([ucl, mk], ignore_index=True)

# Save to final.csv
final.to_csv("final.csv", index=False)

print("✅ final.csv created successfully with all expanded fields.")


✅ final.csv created successfully with all expanded fields.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# Load the final merged dataset
data = pd.read_csv('final.csv')

# Drop rows with missing target
data = data[data['writing score'] != "Missing"]

# Convert scores to numeric
data['writing score'] = pd.to_numeric(data['writing score'], errors='coerce')
data['math score'] = pd.to_numeric(data['math score'], errors='coerce')
data['reading score'] = pd.to_numeric(data['reading score'], errors='coerce')

# Drop rows with missing values in numerical target or features
data.dropna(subset=['writing score', 'math score', 'reading score'], inplace=True)

# Select features
features = ['gender', 'studytime', 'failures', 'internet', 'Extra', 'math score', 'reading score']
X = data[features]
y = data['writing score']

# Fill missing in features
X = X.fillna('Missing')

# Identify categorical and numeric columns
categorical = X.select_dtypes(include='object').columns.tolist()
numerical = X.select_dtypes(include=np.number).columns.tolist()

# Preprocessing
preprocessor = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown='ignore'), categorical),
    ("num", StandardScaler(), numerical)
])

# ML Pipeline
model = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate
print("R² Score:", r2_score(y_test, y_pred))
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))


R² Score: 0.9897954974137438
Mean Squared Error: 9.757813558302088


In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X, y, cv=5, scoring='r2')
print("Cross-validated R² scores:", scores)
print("Average R² score:", scores.mean())


Cross-validated R² scores: [0.78501371 0.73062734 0.94173573 0.93401872 0.74037464]
Average R² score: 0.8263540249033342


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'regressor__n_estimators': [50, 100, 200],
    'regressor__max_depth': [None, 5, 10],
}

grid = GridSearchCV(model, param_grid, cv=5, scoring='r2')
grid.fit(X, y)

print("Best R² Score:", grid.best_score_)
print("Best Parameters:", grid.best_params_)


Best R² Score: 0.8483232801870078
Best Parameters: {'regressor__max_depth': 5, 'regressor__n_estimators': 200}
